<a href="https://colab.research.google.com/github/Ekissi/Ekissi/blob/main/cnn_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue May 21 17:54:30 2024

@author: ASUS
"""


from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing import image
import os
import re
from PIL import Image
import h5py
from tensorflow.keras.models import Sequential
import tensorflow as tf
import pandas as pd
import glob
import fidle
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import  Dense, Dropout, Input, \
     Flatten, GlobalAveragePooling2D

def extract_number(filename):
    """
    Extract the number from a filename.
    Assumes the filename contains numbers, e.g., 'image123.jpg'.
    """
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else -1

def list_files_and_directories_sorted(directory_path):
    try:
        # Get the list of all files and directories
        items = os.listdir(directory_path)
        # Sort items by numbers in descending order
        items.sort(key=extract_number, reverse=False)
        return items
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

def csv_file(image_dir):
    items = list_files_and_directories_sorted(image_dir)
    labels = pd.read_excel('MC2_ok.xlsx') # Example labels, should be of the same length as filenames
    # Create a list of dictionaries for each image and its label
    data = [{'filename': filename, 'label': label} for filename, label in zip(items, labels)]
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    # Save the DataFrame to a CSV file
    df.to_csv('data.csv', index=False)
    return df
#data1 = csv_file(image_dir).iloc[range(22)]
#csv_file(image_dir)
#data = pd.read_csv(image_dir)
def split_folder(image_dir,rand_st):
    train1_data, test1_data = train_test_split(csv_file(image_dir).iloc[range(22)], test_size=0.2, random_state=rand_st)
    train2_data, test2_data = train_test_split(csv_file(image_dir).iloc[range(22,47)], test_size=0.2, random_state=rand_st)
    train3_data, test3_data = train_test_split(csv_file(image_dir).iloc[range(47,69)], test_size=0.2, random_state=rand_st)
    train4_data, test4_data = train_test_split(csv_file(image_dir).iloc[range(69,85)], test_size=0.2, random_state=rand_st)
    train5_data, test5_data = train_test_split(csv_file(image_dir).iloc[range(85,103)], test_size=0.2, random_state=rand_st)
    train6_data, test6_data = train_test_split(csv_file(image_dir).iloc[range(103,119)], test_size=0.2, random_state=rand_st)
    train7_data, test7_data = train_test_split(csv_file(image_dir).iloc[range(119,133)], test_size=0.2, random_state=rand_st)
    train8_data, test8_data = train_test_split(csv_file(image_dir).iloc[range(133,151)], test_size=0.2, random_state=rand_st)
    train9_data, test9_data = train_test_split(csv_file(image_dir).iloc[range(151,170)], test_size=0.2, random_state=rand_st)
    train10_data, test10_data = train_test_split(csv_file(image_dir).iloc[range(170,209)], test_size=0.2, random_state=rand_st)

    train1_data.to_csv('train_data_folder\\train1_data.csv', index=False)
    train2_data.to_csv('train_data_folder\\train2_data.csv', index=False)
    train3_data.to_csv('train_data_folder\\train3_data.csv', index=False)
    train4_data.to_csv('train_data_folder\\train4_data.csv', index=False)
    train5_data.to_csv('train_data_folder\\train5_data.csv', index=False)
    train6_data.to_csv('train_data_folder\\train6_data.csv', index=False)
    train7_data.to_csv('train_data_folder\\train7_data.csv', index=False)
    train8_data.to_csv('train_data_folder\\train8_data.csv', index=False)
    train9_data.to_csv('train_data_folder\\train9_data.csv', index=False)
    train10_data.to_csv('train_data_folder\\train10_data.csv', index=False)

    test1_data.to_csv('test_data_folder\\test1_data.csv', index=False)
    test2_data.to_csv('test_data_folder\\test2_data.csv', index=False)
    test3_data.to_csv('test_data_folder\\test3_data.csv', index=False)
    test4_data.to_csv('test_data_folder\\test4_data.csv', index=False)
    test5_data.to_csv('test_data_folder\\test5_data.csv', index=False)
    test6_data.to_csv('test_data_folder\\test6_data.csv', index=False)
    test7_data.to_csv('test_data_folder\\test7_data.csv', index=False)
    test8_data.to_csv('test_data_folder\\test8_data.csv', index=False)
    test9_data.to_csv('test_data_folder\\test9_data.csv', index=False)
    test10_data.to_csv('test_data_folder\\test10_data.csv', index=False)
    return []

def associat_csv(chemin_train_csv, chemin_test_csv):
    # Lire tous les fichiers CSV dans une liste de DataFrames
    fichiers_csv = glob.glob(chemin_train_csv)
    liste_dataframes = [pd.read_csv(fichier) for fichier in fichiers_csv]
    # Combiner tous les DataFrames en un seul
    df_combine = pd.concat(liste_dataframes, ignore_index=True)
    # Enregistrer le DataFrame combiné dans un nouveau fichier CSV
    df_combine.to_csv('train_data.csv', index=False)

    fichiers_csv = glob.glob(chemin_test_csv)
    liste_dataframes = [pd.read_csv(fichier) for fichier in fichiers_csv]
    # Combiner tous les DataFrames en un seul
    df_combine = pd.concat(liste_dataframes, ignore_index=True)
    # Enregistrer le DataFrame combiné dans un nouveau fichier CSV
    df_combine.to_csv('test_data.csv', index=False)


class DataGenerator(keras.utils.Sequence):
    def _init_(self, data, batch_size = 47, image_size=(2448, 2048), n_channels=3,shuffle=False, preprocess=None, list_IDs=[], db_path=''):
        'Initialization'
        self.preprocess = preprocess
        self.data = data
        self.list_IDs_temp = []
        self.image_size = image_size
        self.db_path = db_path
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def _len_(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.data)/self.batch_size))

    def _getitem_(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        # Find list of IDs
        batch_data = self.data.iloc[indexes]
        # Generate data
        X,y = self.__data_generation(batch_data)
        return X,y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.data))
        if self.shuffle == False:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.image_size, self.n_channels))
        y = np.empty((self.batch_size), dtype=float)
        # Generate data
        for j, (filename, label) in enumerate(zip(batch_data['filename'], batch_data['label'])):
            # img=sklearn.datasets.load_sample_image(self.db_path+ID)
            id = os.path.join(self.db_path, filename)
            img = image.load_img(id, target_size=self.image_size)
            img = image.img_to_array(img)
            img = self.preprocess(img)
            # x = np.expand_dims(img, axis=0)
            X[j, :, :, :] = img
            y[j] = label
        return X,y

# import skimage
def Base_Model(base, weights='imagenet', include_top=False, input_shape=(2448, 2048, 3)):
    if (base == 'resnet50'):
        return ResNet50(weights=weights, include_top=include_top, input_shape=input_shape)
    if (base == 'vgg16'):
        return VGG16(weights=weights, include_top=include_top, input_shape=input_shape)
#base = 'resnet50'
# model = Base_Model(base, weights='imagenet', include_top=False, input_shape=(2448, 2048, 3))

def Feature_Extraction():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(2448, 2048, 3))
    x = base_model.layers[-1].output
    x1 = GlobalAveragePooling2D()(x)
    vgg16_model = keras.Model(inputs=base_model.layers[0].output, outputs=x1)
    print("Model archtecture's details:")
    #vgg16_model.summary()
    return vgg16_model

def Dense_model(input,dropOutRate=0.25,hidden_units=1024,num_layers=3):
       x2=Input(shape=input)
       x2_=Flatten()(x2)
       for i in range(num_layers):
        x2_= Dense(hidden_units,activation='relu',kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),bias_regularizer=l2(0.5))(x2_)
        x2_=Dropout(dropOutRate)(x2_)
       x3=Dense(1,activation="linear")(x2_)
       model=keras.Model(inputs=x2,outputs=x3)
       return model

#[10,20,30,40,50,60,70,80,90,100,150,200,250,300,350,400,450,500,550,600]
random_states =[10,20]
mse_test_list = []
mse_training_list = []
mse_valid_list = []
mae_test_list = []
mae_training_list = []
mae_valid_list = []
Rsquared_test_list = []
Rsquared_training_list = []
for rand_st in random_states:
    chemin_test_csv = 'test_data_folder\\*.csv'
    chemin_train_csv = 'train_data_folder\\*.csv'
    image_dir = 'imagebatch'
    split_folder(image_dir,rand_st)
    associat_csv(chemin_train_csv, chemin_test_csv)
    train_data = pd.read_csv('train_data.csv')
    test_data = pd.read_csv('test_data.csv')
    train_generator = DataGenerator(train_data, image_dir)
    test_generator = DataGenerator(test_data, image_dir)
    base = 'vgg16'
    model = Sequential([
    Feature_Extraction(),
    Flatten(),
    Dense_model((512,))
    ])
    model.summary()

    #Add callback
    os.makedirs('.\DEEP_LEARNING', mode=0o750, exist_ok=True)
    save_dir = "best_model.keras"
    savemodel_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_dir, verbose=0,save_best_only=True)
    model.compile(optimizer = "adam",
                          loss      = "mse",
                          metrics   = ["mae", "mse"])
    history = model.fit(train_generator,
                            epochs = 150,
                            batch_size = 16,
                            validation_data = test_generator,
                            callbacks = [savemodel_callback])
    # Model evaluation
    score = model.evaluate(test_generator, verbose = 0)

    print('x_test/loss       : {:5.4f}'.format(score[0]))
    print('x_test/mae       : {:5.4f}'.format(score[1]))
    print('x_test/mse      : {:5.4f}'.format(score[2]))

    print('min(val_mae) : {:5.4f}'.format(min(history.history['val_mae'])))

    fidle.scrawler.history(history, plot ={'MSE' : ['mse', 'val_mse'],
                                           'MAE' : ['mae', 'val_mae'],
                                           'LOSS' :['loss', 'val_loss']}, save_as = '01-history')
    mse_tt = float(format(score[2]))
    mae_tt = float(format(score[1]))
    mse_test_list.append(mse_tt)
    mae_test_list.append(mae_tt)
    # Restore model
    loaded_model = tf.keras.models.load_model('best_model.keras')
    loaded_model.summary()
    print("Loaded.")

    #Evaluate model
    score= loaded_model.evaluate(test_generator, verbose = 0)

    print('x_test/loss       : {:5.4f}'.format(score[0]))
    print('x_test/mae       : {:5.4f}'.format(score[1]))
    print('x_test/mse      : {:5.4f}'.format(score[2]))
    mse_vd = float(format(score[2]))
    mae_vd = float(format(score[1]))
    mse_valid_list.append(mse_vd)
    mae_valid_list.append(mae_vd)
average_mse_tt = np.mean(mse_test_list)
standard_deviation_mse_tt = np.std(mse_test_list)

average_mae_tt = np.mean(mae_test_list)
standard_deviation_mae_tt = np.std(mae_test_list)

average_mse_vd = np.mean(mse_valid_list)
standard_deviation_mse_vd = np.std(mse_valid_list)

average_mae_vd = np.mean(mae_test_list)
standard_deviation_mae_vd = np.std(mae_valid_list)

print("TEST MSE:","               Average:", average_mse_tt,"","      SD :",standard_deviation_mse_tt)
print("TEST MAE:","               Average:", average_mae_tt,"","       SD :",standard_deviation_mae_tt)

print("VALIDATION MSE:","         Average:", average_mse_vd,"","       SD :",standard_deviation_mse_vd)
print("VALIDATION MAE:","         Average:", average_mae_vd,"","       SD :",standard_deviation_mae_vd)

#%%
#Make prediction
n=len(y_test)
#◘ii=np.random.randint(1,len(X_test),n)
#x_sample = X_test[ii]
#y_sample = y_test[ii]

y_pred = loaded_model.predict(test_features, verbose = 2)

print('MC prediction')
for i in range(n):
    pred  = y_pred[i][0]
    real = y_test[i]
    delta = real-pred
    print(f' {i:03d}       {pred:.2f}         {real}         {delta:+.2f}')


# for m in range(1,6):
fd = os.listdir('imagebatch')
aa = len(fd)
b = aa + 1
generator = DataGenerator(os.listdir('imagebatch'),'imagebatch')
features = np.empty((aa, 2048))
for i in range(1, b):
    # img=sklearn.datasets.load_sample_image(self.db_path+ID)
    id = os.path.join('imagebatch', 'image' + str(i) + '.jpg')
    img = image.load_img(id, target_size=(2448, 2048))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    f = model(img)
    features[i - 1, :] = f
h5f = h5py.File('features_resnet50_ok.h5', 'w')
h5f.create_dataset('features', data=features)
h5f.close()
# del feat_test

# features = model.predict(generator)
# np.save('features_vgg16.npy', features)

# for m in range(1,6):
# fd = os.listdir('training' + str(m))
# aa=len(fd)
#  b =aa+1
#  generator=DataGenerator(batch_size=1, dim=(2448,2048), n_channels=3,
#     shuffle=False,preprocess=preprocess_input,list_IDs=os.listdir('training' + str(m)),db_path='training' + str(m))
#  feat_train = np.empty((aa,2048))
# for  i in range(1,b):
# img=sklearn.datasets.load_sample_image(self.db_path+ID)
# id =os.path.join('training' + str(m), 'image'+str(i)+'.jpg')
# img = image.load_img(id, target_size=(2448,2048))
#    img = image.img_to_array(img)
# img = preprocess_input(img)
# img = np.expand_dims(img, axis=0)
# f = model(img)
# feat_train[i-1,:]=f
# h5f = h5py.File('Training_features_resnet50'+str(m)+'.h5', 'w')
# h5f.create_dataset('training_features', data=feat_train)
# h5f.close()
# del feat_train